In [27]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup  #HTML 파싱
from urllib.parse import quote  #검색어 인코딩
import time  #슬립 사용
import requests  #HTML 가져오기
import datetime

In [28]:
#성인물 포함 데이터
contents_movie = pd.read_csv('./LG_data/01. 전처리/contents_movie_성인물 포함.csv')

print(contents_movie.info())
print(contents_movie.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37183 entries, 0 to 37182
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      37183 non-null  int64  
 1   subsr           37183 non-null  int64  
 2   series_nm       0 non-null      float64
 3   super_asset_nm  37183 non-null  object 
 4   ct_cl           37183 non-null  object 
 5   genre_of_ct_cl  37183 non-null  object 
 6   SMRY            37183 non-null  object 
 7   ACTR_DISP       37183 non-null  object 
 8   disp_rtm        37183 non-null  object 
 9   log_dt          37183 non-null  object 
 10  type            37183 non-null  object 
 11  hour            37183 non-null  int64  
 12  day_of_week     37183 non-null  int64  
 13  is_holiday      37183 non-null  bool   
dtypes: bool(1), float64(1), int64(4), object(8)
memory usage: 3.7+ MB
None
   Unnamed: 0     subsr  series_nm super_asset_nm ct_cl genre_of_ct_cl  \
0           0  617790

시계열 변환 - log_dt : date / time

In [29]:
log_date = []
log_time = []

for value in contents_movie['log_dt'].astype(str) :
  date_data = datetime.datetime.strptime(value, '%Y-%m-%d %H:%M:%S').date()
  time_data = datetime.datetime.strptime(value, '%Y-%m-%d %H:%M:%S').time()
  log_date.append(date_data)
  log_time.append(time_data)

contents_movie['log_date'] = log_date
contents_movie['log_time'] = log_time

running_time : time만

In [30]:
running_data = []
for value in contents_movie['disp_rtm'] :
  time_data = datetime.datetime.strptime(value, '%Y-%m-%d %H:%M:%S').time()
  running_data.append(time_data)
  
contents_movie['disp_rtm'] = running_data

In [31]:
contents_movie.head()


,Unnamed: 0,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt,type,hour,day_of_week,is_holiday,log_date,log_time
0,0,61779000,NaN,마들렌,영화,멜로,소설가를 꿈꾸는 지석. 헤어 디자이너를 꿈꾸는 희진. 몇 번의 우연한 만남이 이어지...,"조인성,신민아,박정아,강래연,김수로,최규환",01:56:00,2024-01-24 22:12:13,movie,1,2,False,2024-01-24,22:12:13
1,1,67170000,NaN,1번가의 기적,영화,코미디,"재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....","임창정,하지원,주현,정두홍,이훈,강예원,박창익,박유선",01:52:00,2024-01-05 09:08:08,movie,1,4,False,2024-01-05,09:08:08
2,2,66547000,NaN,1번가의 기적,영화,코미디,"재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....","임창정,하지원,주현,정두홍,이훈,강예원,박창익,박유선",01:52:00,2024-01-06 01:34:07,movie,1,5,True,2024-01-06,01:34:07
3,3,66547000,NaN,1번가의 기적,영화,코미디,"재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....","임창정,하지원,주현,정두홍,이훈,강예원,박창익,박유선",01:52:00,2024-01-06 03:26:58,movie,1,5,True,2024-01-06,03:26:58
4,4,64300000,NaN,1번가의 기적,영화,코미디,"재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....","임창정,하지원,주현,정두홍,이훈,강예원,박창익,박유선",01:52:00,2024-01-06 23:51:13,movie,1,5,True,2024-01-06,23:51:13


In [32]:
contents_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37183 entries, 0 to 37182
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      37183 non-null  int64  
 1   subsr           37183 non-null  int64  
 2   series_nm       0 non-null      float64
 3   super_asset_nm  37183 non-null  object 
 4   ct_cl           37183 non-null  object 
 5   genre_of_ct_cl  37183 non-null  object 
 6   SMRY            37183 non-null  object 
 7   ACTR_DISP       37183 non-null  object 
 8   disp_rtm        37183 non-null  object 
 9   log_dt          37183 non-null  object 
 10  type            37183 non-null  object 
 11  hour            37183 non-null  int64  
 12  day_of_week     37183 non-null  int64  
 13  is_holiday      37183 non-null  bool   
 14  log_date        37183 non-null  object 
 15  log_time        37183 non-null  object 
dtypes: bool(1), float64(1), int64(4), object(10)
memory usage: 4.3+ MB


In [35]:
for idx in contents_movie.index :
  contents_movie.loc[idx, 'super_asset_nm'] = re.sub('[.]', '', contents_movie.loc[idx, 'super_asset_nm'])

contents_movie.head()

,Unnamed: 0,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt,type,hour,day_of_week,is_holiday,log_date,log_time
0,0,61779000,NaN,마들렌,영화,멜로,소설가를 꿈꾸는 지석. 헤어 디자이너를 꿈꾸는 희진. 몇 번의 우연한 만남이 이어지...,"조인성,신민아,박정아,강래연,김수로,최규환",01:56:00,2024-01-24 22:12:13,movie,1,2,False,2024-01-24,22:12:13
1,1,67170000,NaN,1번가의 기적,영화,코미디,"재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....","임창정,하지원,주현,정두홍,이훈,강예원,박창익,박유선",01:52:00,2024-01-05 09:08:08,movie,1,4,False,2024-01-05,09:08:08
2,2,66547000,NaN,1번가의 기적,영화,코미디,"재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....","임창정,하지원,주현,정두홍,이훈,강예원,박창익,박유선",01:52:00,2024-01-06 01:34:07,movie,1,5,True,2024-01-06,01:34:07
3,3,66547000,NaN,1번가의 기적,영화,코미디,"재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....","임창정,하지원,주현,정두홍,이훈,강예원,박창익,박유선",01:52:00,2024-01-06 03:26:58,movie,1,5,True,2024-01-06,03:26:58
4,4,64300000,NaN,1번가의 기적,영화,코미디,"재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....","임창정,하지원,주현,정두홍,이훈,강예원,박창익,박유선",01:52:00,2024-01-06 23:51:13,movie,1,5,True,2024-01-06,23:51:13


성인물 / 키즈 제거

In [36]:
contents_movie = contents_movie.drop(contents_movie[(contents_movie['ct_cl'] == '키즈') | (contents_movie['genre_of_ct_cl'] == '성인')
                                                  | ((contents_movie['ct_cl'] == '기타') & (contents_movie['genre_of_ct_cl'] == '기타'))
                                                  | ((contents_movie['ct_cl'] == '영화') & (contents_movie['genre_of_ct_cl'] == '기타'))].index, 
                    axis= 0)

contents_movie.columns = ['Unnamed: 0', 'ID', 'series_nm', 'title', 'category', 'genre_category',
                          'SMRY', 'cast', 'running_time', 'log_dt', 'type', 'hour', 'day_of_week', 'is_holiday',
                          'log_date', 'log_time']            

성인물을 제외한 contents_movie.csv 생성

In [37]:
contents_movie.to_csv('./LG_data/01. 전처리/contents_movie_전처리.csv', header= True,
                    mode= 'a', encoding= 'cp949')